## BCI Competition 4 Dataset 2a Results

I show the evolution of the 5-layer performance across changing:
* number of filters
* filter lengths
* avg pooling in final layer instead of fully connected
* dropout on input
* tied losses for samples of same trial
* squared activations

The original values for the samplewise trained model were:

|Highpass|Test|
|-|
|-|**65.6%**|
|4|**61.7%**|

### Number of filters

* Doubling after pooling increases ~3% 

Originally, we had 40 units per layer, we tried:
  * 20,40,80,160
  * 25,50,100,200
  * 32,48,72,108
  * 32,64,128,256
  

|#Units|Highpass|Test|
|-|
|20,40,80,160|-|68.1%|
|20,40,80,160|4|63.1%|
|25,50,100,200|-|**69.5%**|
|25,50,100,200|4|**64.0%**|
|32,48,72,108|-|63.9%|
|32,48,72,108|-|63.8%|
|32,64,128,256|-|68.6%|
|32,64,128,256|-|65.9%|

I continued with 25,50,100,200 since its faster and close to 32,64,128,256. Also I made a mistake (evaluating an experiment which had not finished on all datasets) which led me to believe the 32,64,128,256 is less robust... neverthless at the end of this notebook, I recompare the final model to 32,64,128,256 and there are no big improvements with 32,64,128,256


### Filter lengths

* 15,7,3,3 is >5% worse than 10,10,10,10

I made some smaller experiments with different filter lengths (15,7,3,3 compared to 10,10,10,10) in later layers which all had significantly worse results (< 60%). Might have to try more variants for a fairer comparison. 

### AvgPool + Dropout

* Average pool helps a little on highpassed data
* same for dropout

Using 25,50,100,200, we got:

|Highpass|Dropout in|Test|
|-|
|-|-|67.6%|
|4|-|65.1%|
|-|0.1|**67.5%**|
|4|0.1|**66.4%**|
|-|0.3|68.1%|
|4|0.3|66.0%|

I resumed with dropout 0.1 (**in bold**).

## Factorized

* modest increase without highpass, very modest decrease with highpass -> inconclusive, not pursued further

Did not help much. Note that factorization of for example 11x1 into 6x1+6x1 increases the number of weights (in contrast with 2d-filters where 11x11 has more parameters than 6x6+6x6).

|Highpass|nonlin between|Test|
|-|
|-|-|70.7%|
|4|-|65.9%|
|-|elu|68.0%|
|4|elu|63.3%|


## Tied Losses

* Tying neighbouring losses increases ~1%

I tested either using 1,10 or 100 randomly chosen pairs (chosen so that any sample maximally appears once) or using all neighbouring samples.


|Highpass|Tied pairs|Drop in prob|Test|
|-|
|-|1|0.0|65.8%|
|-|10|0.0|68.0%|
|-|100|0.0|67.8%|
|-|neighb|0.0|**69.7%**|
|4|1|0.0|62.5%|
|4|10|0.0|66.7%|
|4|100|0.0|67.4%|
|4|neighb|0.0|**67.7%**|
|-|1|0.1|64.2%|
|-|10|0.1|66.8%|
|-|100|0.1|67.2%|
|-|neighb|0.1|70.2%|
|4|1|0.1|60.7%|
|4|10|0.1|64.3%|
|4|100|0.1|65.7%|
|4|neighb|0.1|65.7%|

## Removing avg pool

* Removing avg pool very modst decrease for highpass, modest increase for no highpass -> used due to more conceptual simplicity of the model (no special avg pooling layer at end)

|Highpass|Tied pairs|Drop in prob|Test|
|-|
|-|neighb|0.0|**71.0%**|
|4|neighb|0.0|**67.2%**|

### Our data comparison

* Removing avg makes bug problems >3% worsening on our data without highpass, therefore retried our data with resampling

unfortunately removing avg is a big problem here for one  :(

|Highpass|Tied pairs|#units|avg|Test|
|-|
|-|no|40,40,40,40|no|90.6%|
|4|no|40,40,40,40|no|86.7%|
|-|neighb|25,50,100,200|yes|89.9%|
|4|neighb|25,50,100,200|yes|89.0%|
|-|neighb|25,50,100,200|no|86.8%|
|4|neighb|25,50,100,200|no|89.2%|


## Our data 250 Hz

* Resampling leads to same results as before with avgpool -> final model for both bci competition and our data


|Highpass|1st Filter Length|Tied pairs|#units|avg|Test|
|-|
|-|neighb|10|25,50,100,200|no|**89.3%/89.4%**|
|4|neighb|10|25,50,100,200|no|**88.9%/89.2%**|
|-|neighb|10|32,64,128,256|no|90.3%|
|4|neighb|10|32,64,128,256|no|88.1%|
|-|neighb|25|25,50,100,200|no|89.6%|
|4|neighb|25|25,50,100,200|no|86.9%|
|-|neighb|25|32,64,128,256|no|90.4%|
|4|neighb|25|32,64,128,256|no|86.9%|

The two values for the final configuration are from two different, supposedly identical runs. So difference is either due to CuDNNs' inherent indeterminism or some bug of mine.




## Square auf 250 Hz

* Squaring is not helpful in any way, if only on first layer atleast performance is similar


|Highpass|Tied pairs|#units|avg|square layers|Test|
|-|
|-|neighb|25,50,100,200|no|first|86.5%|
|4|neighb|25,50,100,200|no|first|88.7%|
|-|neighb|25,50,100,200|no|all|84.3%|
|4|neighb|25,50,100,200|no|all|83.1%|

## Final model choice

So the final model choice is:
* 25,50,100,200 filters
* all filter lengths 10
* no dropout on input
* tied neighbour losses
* no squaring
* no avg pooling
* resampling our data to 250 Hz


### Our Data

|Highpass|Test|
|-|
|-|**89.3%/89.4%**|
|4|**88.9%/89.2%**|


### BCI Competition Data

|Highpass|Test|
|-|
|-|**71.0%**|
|4|**67.2%**|
